In [1]:
import numpy as np
import json
import os

from util import Constants
import util

In [2]:
# EXAMPLE INPUT FILE
input_file = "processed_sheets/Amy Winehouse, Winehouse - YOU KNOW I'M NO GOOD.json"

with open(input_file, 'r') as f:
    sheet = json.load(f)
    
# SHEET SCHEMA
# {'note': {'pitch': int, 'alter': int, 'rest': boolean},
#   'rhythm': {'duration': float,
#    'divisions': float,
#    'index': int,
#    'new_measure': boolean},
#   'chord': {'root': String, 'mode': String, 'alter': int}}

In [3]:
# CONVERT TO NUMPY REPRESENTATION

# NUMPY SCHEMA
# [0:127] -> MIDI pitch (one-hot)
# [128] -> rest (one-hot)
# [129] -> barline (one-hot)

# [130:141] -> rhythm index (one-hot)
# [142] -> barline (one-hot) (repeated)

# [143:190] -> chord (one-hot)
# [191] -> barline (one-hot) (repeated)

D = 192
np_sheet = []

for x in sheet:
    if x['rhythm']['new_measure']:
        entry = np.zeros(192)
        entry[128] = 1
        entry[142] = 1
        entry[191] = 1
        np_sheet.append(entry)
    
    entry = np.zeros(192)
    
    entry[0 + x['note']['pitch']] = 1
    entry[128] = int(x['note']['rest'])
    entry[130 + x['rhythm']['index']] = 1
    
    chord_index = util.chord_to_index(x['chord']['root'], x['chord']['mode'], x['chord']['alter'])
    entry[143 + chord_index] = 1
    
    np_sheet.append(entry)

In [4]:
np.asarray(np_sheet)

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])